# Get related info for the SGCN species pages

The [species view](https://www1.usgs.gov/csas/swap/species_view.html) pages in the SWAP app have a section that pulls information from USFWS and NatureServe. In the current app, the NatureServe information is coming through a PHP page that looks up information from NatureServe web services. I haven't looked into how the Federal listing status information is coming into the application.

I used this opportunity to start exercising what we are calling our Taxonomic Information Registry within the Biogeographic Information System - a capability that takes all of our taxa of interest and assembles information for use. I built a basic triple store capability called "tircache" to house information about SGCN taxa needed by the SWAP app and wrote a couple of notebooks to go get information from FWS via the [ECOS TESS service](https://ecos.fws.gov/ecp/species-query) and the [NatureServe web services](http://services.natureserve.org/). We eventually need to move tircache to a proper triple store, but for now I stuck it into a table within the GC2 platform that we are exercising for other stuff.

The tircache data are structured with the following:

* Subject (subjectid) - identifier that is currently either a taxonomic authority ID from ITIS or WoRMS or a species scientific name (see below)
* Predicate (property) - so far, a descriptive name that indicates the type of information associated with the subject ID (e.g., "TESS:StatusText" for the current listing status of a species)
* Object (value) - the value of the property (E.g., "Endangered" returned from the ECOS TESS service for Federal listing status)

Queries against the GC2 API can be used to retrieve this information for the SWAP app in lieu of the current methods of going after the information directly from what are somewhat difficult and unreliable services to work with. When retrieving information into tircache, the processes first figure out if there are applicable records and then insert either a positive or negative response on the subject ID with a date/time that the information was checked. We will be setting this system up to run on some kind of scheduled basis via a set of microservices off the Kafka queue once we have that in place.

In the SGCN case, Abby originally ran the taxonomic lookup by species scientific names to get an associated ITIS identifier, following the ITIS taxonomic information to get the currently accepted identifier in some cases. Because of how we originally loaded the data, we ended up storing the ITIS TSN in the SGCN data. From that point, we can use that ID to go get other associated information from ITIS and ECOS TESS and cache it for use in tircache. With NatureServe, we needed to use the scientific name (already spruced up from ITIS in some cases) to retrieve NatureServe's "Element Global UID," from which we can go after more information. A better way to handle this in future is probably to just start with whatever species identification is provided by source data, run processes to use that in finding other persistent identifiers of interest, and then use those persistent identifiers in going after more information.

This notebook shows the logical queries that can be run using the GC2 API to retrieve FWS listing and NatureServe information for use in the SWAP app.